# 불러오기

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn

In [ ]:
sklearn.__version__

In [35]:
train =  pd.read_csv('data/preprocess/train_0916.csv')
test =  pd.read_csv('data/preprocess/test_0916.csv')

In [ ]:
train['MasVnrArea'].hist()

In [48]:
train=train.fillna(0)
test=test.fillna(0)

In [37]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,
                                                  test_size=0.3,
                                                  random_state=2045)

print('Train Data : ',X_train.shape,y_train.shape)
print('Test Data : ',X_test.shape,y_test.shape)

Train Data :  (1022, 51) (1022,)
Test Data :  (438, 51) (438,)


In [ ]:
X_train.info()

In [ ]:
y_train.info()

# multivaricable regression

In [12]:
from sklearn.linear_model import LinearRegression

MR=LinearRegression()
MR.fit(X_train,y_train)

LinearRegression()

In [13]:
print('weight(w) : ',MR.coef_)
print('bias(b) : ','%.8f' % MR.intercept_)

weight(w) :  [[-1.96947016e+03 -2.37595232e+02  6.64123333e+04  3.14456713e+04
  -1.46809495e+03  6.71764403e+03  2.03205292e+02  1.09874283e+04
   5.27523035e+02 -2.35736952e+02 -1.77541285e+02 -8.53503053e+02
   1.26288686e+04  3.43819775e+03  2.83234455e+03  3.88542999e+03
   5.12241598e+04 -9.03387276e+03  9.76072621e+02  2.71576490e+03
  -9.68445581e+03  3.30305888e+03 -5.33301641e+03 -2.78762708e+03
  -5.95779735e+02  2.77830621e+03  5.94774072e+02  2.67253813e+05
   1.12673115e+04  5.17128476e+03  7.76004534e+03  4.93825370e+03
  -1.53757304e+03 -6.68520017e+03 -9.85141636e+03  5.06721603e+03
   6.95890051e+03  2.47724191e+01  3.71647239e+02  3.54814399e+04
  -3.17305835e+02  3.17515197e+03  2.66423109e+03  2.14628981e+04
  -2.00917963e+04 -1.18185775e+04  7.77854648e+03  1.76667707e+04
   2.03787927e+02 -2.37423342e+02  1.29691991e+03]]
bias(b) :  -47797.46194242


In [14]:
MR.score(X_test,y_test)

0.8654866809526802

In [31]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score

print(mean_squared_error(y_test, MR.predict(X_test)))
print(r2_score(y_test, MR.predict(X_test)))
print(mean_squared_log_error(y_test, MR.predict(X_test)))
print(np.sqrt(mean_squared_log_error(y_test, MR.predict(X_test))))

633098101.4603996
0.8654866809526802
0.027247104025936893
0.16506696830661455


# Polynomial regression

In [18]:
from sklearn.preprocessing import PolynomialFeatures

poly=PolynomialFeatures(degree=3, include_bias=False)
X_train_poly=poly.fit_transform(X_train)

print('변환 전 데이터: ',X_train.shape)
print('2차항 변환 데이터: ', X_train_poly.shape)

변환 전 데이터:  (1022, 51)
2차항 변환 데이터:  (1022, 24803)


In [19]:
from sklearn.linear_model import LinearRegression

NL=LinearRegression()
NL.fit(X_train_poly,y_train)

LinearRegression()

In [20]:
print('weight(w) : ',NL.coef_)
print('bias(b) : ','%.8f' % NL.intercept_)

weight(w) :  [[ -6.42203322   0.92634665   1.47016174 ... -86.30474159  33.12681907
  -44.09759721]]
bias(b) :  79149.02536621


In [27]:
X_test_poly=poly.fit_transform(X_test)

NL.score(X_test_poly, y_test)

0.21537468966768003

In [30]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score

X_test_poly=poly.fit_transform(X_test)

print(mean_squared_error(y_test, NL.predict(X_test_poly)))
print(r2_score(y_test, NL.predict(X_test_poly)))

3692904151.4054184
0.21537468966768003


# RandomForestRegressor

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split

# RMSLE 정의
def rmsle(origin, pred):
    return np.sqrt(mean_squared_log_error(origin, pred))

# X, y를 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1)

### Train
- 우리가 가진 데이터셋 내에서 훈련 -> 예측 -> 검증을 해보면서 모델의 성능을 평가/향상시키려고 함
- 그래서 가격을 알고 있는 데이터(여기선 X,y)를 7:3 비율로 train : test로 쪼개서 우리가 돌리는 모델을 검증해보려고 함
- 근데 아직은 K-Fold Validation이라던가 유의미한 방법은 적용 안해봄.  

그래서 아직은
- 큰 의미가 있는 과정이라기보다는, RandomForestRegressor가 잘 돌아가는구나~
- 우리가 가진 데이터 내에서 검증해봤을때 RMSLE 가 0.012면 꽤 잘 나오는구나~
- feature importance 관찰용
- 정도로 생각하시면 될 듯 (이제 여러가지 시도해봐야죠 ㅎㅎ)

In [40]:
forest_train = RandomForestRegressor(n_estimators=1000, 
                              criterion='mse',
                              random_state=1,
                              n_jobs=-1)
forest_train.fit(X_train, y_train)
y_train_pred = forest_train.predict(X_train)
y_test_pred = forest_train.predict(X_test)

print('훈련 MSE: %.3f, 테스트 MSE: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('훈련 R^2: %.3f, 테스트 R^2: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))
print('훈련 RMSLE: %.3f, 테스트 RMSLE: %.3f' % (
        rmsle(y_train, y_train_pred),
        rmsle(y_test, y_test_pred)))

훈련 MSE: 148125928.212, 테스트 MSE: 767543418.809
훈련 R^2: 0.975, 테스트 R^2: 0.893
훈련 RMSLE: 0.060, 테스트 RMSLE: 0.160


importance가 낮은 feature drop

In [41]:
importance = forest_train.feature_importances_

In [42]:
del_list = []
for i in range(len(test.columns.tolist())):
    if importance[i] < 0.0001:
        del_list.append(test.columns.tolist()[i])
len(del_list)

1

In [43]:
X = X.drop(columns=del_list,axis=1)
test = test.drop(columns=del_list,axis=1)

In [50]:
forest = RandomForestRegressor(n_estimators=1000, 
                              criterion='mse',
                              random_state=1,
                              n_jobs=-1)
forest.fit(X, y)
pred = forest.predict(test)
pred = np.expm1(pred)

<ipython-input-50-6d7a10c5b150>:7: RuntimeWarning: overflow encountered in expm1
  pred = np.expm1(pred)


In [49]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 50 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1459 non-null   int64  
 2   LotArea        1459 non-null   float64
 3   LotShape       1459 non-null   int64  
 4   LandContour    1459 non-null   int64  
 5   LotConfig      1459 non-null   int64  
 6   LandSlope      1459 non-null   int64  
 7   Neighborhood   1459 non-null   int64  
 8   Condition1     1459 non-null   int64  
 9   BldgType       1459 non-null   int64  
 10  HouseStyle     1459 non-null   int64  
 11  OverallQual    1459 non-null   int64  
 12  OverallCond    1459 non-null   int64  
 13  RoofStyle      1459 non-null   int64  
 14  MasVnrType     1459 non-null   int64  
 15  MasVnrArea     1459 non-null   float64
 16  ExterQual      1459 non-null   int64  
 17  ExterCond      1459 non-null   int64  
 18  Foundati

In [ ]:
pred

- 저장

In [ ]:
# sub=pd.read_csv('data/sample_submission.csv')
# sub['SalePrice']=pred
# sub.to_csv('submission/sub_1_0910.csv',index=False)

# XGBoost
- 이건 아직 복붙해서 돌려본거라...
- 대충 돌려봤는데도 RandomForest보다 훨씬 잘 나옴
- 더 잘 해보고 수정하겠음...

In [ ]:
# from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
# from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
# from sklearn.kernel_ridge import KernelRidge
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import RobustScaler
# from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
# from sklearn.model_selection import KFold, cross_val_score, train_test_split
# from sklearn.metrics import mean_squared_error
# from xgboost import XGBRegressor
# import xgboost as xgb
# import lightgbm as lgb
# from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_regression
# from sklearn.model_selection import GridSearchCV

In [ ]:
# #Validation function
# n_folds = 10

# def rmsle_cv(model):
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
#     rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
#     return(rmse)

In [ ]:
# model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
#                              learning_rate=0.05, max_depth=3, 
#                              min_child_weight=1.7817, n_estimators=2200,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5213, silent=1,
#                              random_state =7, nthread = -1)

In [ ]:
# model_xgb.fit(X_train, y_train)
# xgb_train_pred = model_xgb.predict(X_train)
# xgb_pred = np.expm1(model_xgb.predict(X_test))
# print(rmsle(y_train, xgb_train_pred))
# print(rmsle(y_test, xgb_pred))

In [ ]:
# model_xgb.fit(X, y)
# pred_xgb = model_xgb.predict(test)

In [ ]:
# pred_log_xgb = np.expm1(pred_xgb)

In [ ]:
# sub=pd.read_csv('data/sample_submission.csv')
# sub['SalePrice']=pred_log_xgb
# sub.to_csv('sub_0910_xgb.csv',index=False)

# 앙상블